# 1. Librerias

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 52.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=804e7bcca009a456a198acf6b6f190c3d8f0375fdbcbfca998ce3780665f8752
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [31]:
#%matplotlib inline
#import pandas as pd
#import numpy as np

#Importamos la librería de funciones clasicas
import pyspark.sql.functions as f

#Librería para crear funciones personalizadas
from pyspark.sql.functions import udf, struct

#Importamos los tipos de datos
from pyspark.sql.types import *

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# 2. Lectura de datos

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
"""Trabajaremos con un dataset de tres campos:

1.   Gender: sexo de la persona (Male, Female)
2.   Height: Estatura
3.   Weight: Peso


"""

#Leemos los datos
#Lectura de archivo JSON
#dfJson=spark.read.format("json").option("multiLine", "true").load("/content/drive/MyDrive/Data/transacciones.json")
dfJson=spark.read.format("json").option("multiLine", "false").load("/content/drive/MyDrive/Data/transacciones.json")

In [6]:
#Mostramos los datos
#Vemos que lo está truncando
dfJson.show(5, False)

+------------+------------------------------------------------------------------------------------------------------+--------------------+
|EMPRESA     |PERSONA                                                                                               |TRANSACCION         |
+------------+------------------------------------------------------------------------------------------------------+--------------------+
|{5, Amazon} |{[{59, 9811935}, {53, 9423163}], 33, 26, Brenden, 20549.0}                                            |{2021-01-23, 2628.0}|
|{9, IBM}    |{[{50, 9912937}, {54, 9046676}, {55, 9874284}, {58, 9746053}, {53, 9058704}], 31, 21, Carissa, 1952.0}|{2021-01-23, 4261.0}|
|{7, Samsung}|{[{53, 9769557}, {59, 9754523}, {52, 9063371}, {55, 9301624}, {56, 9770100}], 42, 73, Fiona, 9960.0}  |{2021-01-23, 1429.0}|
|{5, Amazon} |{[{51, 9733329}, {57, 9619332}, {51, 9087416}, {50, 9486747}], 59, 14, Allen, 16289.0}                |{2021-01-23, 3385.0}|
|{4, Toyota} |{[{52, 909133

In [7]:
#Pintamos el esquema-- Trae la semiestructura global
## No necesariamente todos los registros van a tener todos los campos
dfJson.printSchema()

root
 |-- EMPRESA: struct (nullable = true)
 |    |-- ID_EMPRESA: string (nullable = true)
 |    |-- NOMBRE_EMPRESA: string (nullable = true)
 |-- PERSONA: struct (nullable = true)
 |    |-- CONTACTO: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- PREFIJO: string (nullable = true)
 |    |    |    |-- TELEFONO: string (nullable = true)
 |    |-- EDAD: long (nullable = true)
 |    |-- ID_PERSONA: string (nullable = true)
 |    |-- NOMBRE_PERSONA: string (nullable = true)
 |    |-- SALARIO: double (nullable = true)
 |-- TRANSACCION: struct (nullable = true)
 |    |-- FECHA: string (nullable = true)
 |    |-- MONTO: double (nullable = true)



In [13]:
#Estructuramiento de dataframes
#Estructuramos dfEmpresa

#Seleccionamos los campos
dfEmpresa = dfJson.select(
  dfJson["EMPRESA.ID_EMPRESA"].alias("ID_EMPRESA"),
  dfJson["EMPRESA.NOMBRE_EMPRESA"].alias("NOMBRE_EMPRESA")
)

#Vemos el esquema
dfEmpresa.printSchema()

dfEmpresa.show(5, False)

root
 |-- ID_EMPRESA: string (nullable = true)
 |-- NOMBRE_EMPRESA: string (nullable = true)

+----------+--------------+
|ID_EMPRESA|NOMBRE_EMPRESA|
+----------+--------------+
|5         |Amazon        |
|9         |IBM           |
|7         |Samsung       |
|5         |Amazon        |
|4         |Toyota        |
+----------+--------------+
only showing top 5 rows



In [15]:
#Estructuramos dfPersona

#Seleccionamos los campos
dfPersona = dfJson.select(
  dfJson["PERSONA.ID_PERSONA"].alias("ID_PERSONA"),
  dfJson["PERSONA.NOMBRE_PERSONA"].alias("NOMBRE_PERSONA"),
  dfJson["PERSONA.EDAD"].alias("EDAD"),
  dfJson["PERSONA.SALARIO"].alias("SALARIO")
)

#Vemos el esquema
dfPersona.printSchema()

#Mostramos los datos
dfPersona.show(20, False)

root
 |-- ID_PERSONA: string (nullable = true)
 |-- NOMBRE_PERSONA: string (nullable = true)
 |-- EDAD: long (nullable = true)
 |-- SALARIO: double (nullable = true)

+----------+--------------+----+-------+
|ID_PERSONA|NOMBRE_PERSONA|EDAD|SALARIO|
+----------+--------------+----+-------+
|26        |Brenden       |33  |20549.0|
|21        |Carissa       |31  |1952.0 |
|73        |Fiona         |42  |9960.0 |
|14        |Allen         |59  |16289.0|
|80        |Ebony         |59  |3600.0 |
|53        |Zachery       |22  |23820.0|
|31        |Rylee         |47  |21591.0|
|55        |Jennifer      |64  |19013.0|
|45        |Kylynn        |22  |7040.0 |
|22        |Kibo          |22  |7449.0 |
|64        |Graiden       |28  |22037.0|
|90        |Sigourney     |42  |9145.0 |
|59        |Quemby        |26  |12092.0|
|17        |Omar          |60  |6851.0 |
|64        |Graiden       |28  |22037.0|
|61        |null          |33  |15070.0|
|84        |Keith         |33  |13348.0|
|35        |A

In [21]:
#Importamos la función que aplana los campos arrays
from pyspark.sql.functions import explode

#Estructuramos dfContacto

#Seleccionamos los campos
dfContacto1 = dfJson.select(
  dfJson["PERSONA.ID_PERSONA"].alias("ID_PERSONA"),
  f.explode(dfJson["PERSONA.CONTACTO"]).alias("DATOS_CONTACTO")
)

#Vemos el esquema
dfContacto1.printSchema()

#//Mostramos los datos
dfContacto1.show(20, False)

#//Seleccionamos los campos
dfContacto = dfContacto1.select(
  dfContacto1["ID_PERSONA"].alias("ID_PERSONA"),
  dfContacto1["DATOS_CONTACTO.PREFIJO"].alias("PREFIJO"),
  dfContacto1["DATOS_CONTACTO.TELEFONO"].alias("TELEFONO")
)

#//Vemos el esquema
dfContacto.printSchema()

#//Mostramos los datos
dfContacto.show(20, False)

#También podemos hacerlo en un paso encadenando pasos
#dfContacto = dfJson.select(
#  dfJson["PERSONA.ID_PERSONA"].alias("ID_PERSONA"),
#  f.explode(dfJson["PERSONA.CONTACTO"]).alias("DATOS_CONTACTO")
#).alias("P1").select(
#  f.["P1.ID_PERSONA"].alias("ID_PERSONA"),
#  f.["P1.DATOS_CONTACTO.PREFIJO"].alias("PREFIJO"),
#  f.["P1.DATOS_CONTACTO.TELEFONO"].alias("TELEFONO")
#)

#//Vemos el esquema
#dfContacto.printSchema()

#//Mostramos los datos
#dfContacto.show(20, false)

root
 |-- ID_PERSONA: string (nullable = true)
 |-- DATOS_CONTACTO: struct (nullable = true)
 |    |-- PREFIJO: string (nullable = true)
 |    |-- TELEFONO: string (nullable = true)

+----------+--------------+
|ID_PERSONA|DATOS_CONTACTO|
+----------+--------------+
|26        |{59, 9811935} |
|26        |{53, 9423163} |
|21        |{50, 9912937} |
|21        |{54, 9046676} |
|21        |{55, 9874284} |
|21        |{58, 9746053} |
|21        |{53, 9058704} |
|73        |{53, 9769557} |
|73        |{59, 9754523} |
|73        |{52, 9063371} |
|73        |{55, 9301624} |
|73        |{56, 9770100} |
|14        |{51, 9733329} |
|14        |{57, 9619332} |
|14        |{51, 9087416} |
|14        |{50, 9486747} |
|80        |{52, 9091334} |
|80        |{59, 9831571} |
|53        |{59, 9708669} |
|53        |{52, 9751344} |
+----------+--------------+
only showing top 20 rows

root
 |-- ID_PERSONA: string (nullable = true)
 |-- PREFIJO: string (nullable = true)
 |-- TELEFONO: string (nullable =

In [22]:
#Estructuramos dfTransaccion
[]

#Seleccionamos los campos
dfTransaccion = dfJson.select(
  dfJson["PERSONA.ID_PERSONA"].alias("ID_PERSONA"),
  dfJson["EMPRESA.ID_EMPRESA"].alias("ID_EMPRESA"),
  dfJson["TRANSACCION.MONTO"].alias("MONTO"),
  dfJson["TRANSACCION.FECHA"].alias("FECHA")
)

#//Vemos el esquema
dfTransaccion.printSchema()

#//Mostramos los datos
dfTransaccion.show(20, False)

root
 |-- ID_PERSONA: string (nullable = true)
 |-- ID_EMPRESA: string (nullable = true)
 |-- MONTO: double (nullable = true)
 |-- FECHA: string (nullable = true)

+----------+----------+------+----------+
|ID_PERSONA|ID_EMPRESA|MONTO |FECHA     |
+----------+----------+------+----------+
|26        |5         |2628.0|2021-01-23|
|21        |9         |4261.0|2021-01-23|
|73        |7         |1429.0|2021-01-23|
|14        |5         |3385.0|2021-01-23|
|80        |4         |3514.0|2021-01-23|
|53        |9         |823.0 |2021-01-23|
|31        |2         |3724.0|2021-01-23|
|55        |10        |3429.0|2021-01-23|
|45        |4         |4267.0|2021-01-23|
|22        |9         |796.0 |2021-01-23|
|64        |4         |317.0 |2021-01-23|
|90        |9         |938.0 |2021-01-23|
|59        |8         |887.0 |2021-01-23|
|17        |5         |2067.0|2021-01-23|
|64        |7         |714.0 |2021-01-23|
|61        |5         |2967.0|2021-01-23|
|84        |1         |769.0 |2021-01-

In [ ]:
#Obtenemos el primer elemento de cada array
###, o sea va a traer un correo por registro del array
df5 = dfJson.select(dfJson["PERSONA.CONTACTO"].getItem(0))

#Mostramos la data
df5.show(5, False)

+-------------------------------+
|PERSONA.CONTACTO AS CONTACTO[0]|
+-------------------------------+
|{59, 9811935}                  |
|{50, 9912937}                  |
|{53, 9769557}                  |
|{51, 9733329}                  |
|{52, 9091334}                  |
+-------------------------------+
only showing top 5 rows



## Almacenamiento en archivos parquet

In [24]:

#//Los dataframes estructurados se almacenan para que estén listos para el análisis

#//Almacenamos los datos de PERSONA
dfPersona.write.format("parquet").mode("overwrite").option("compression", "snappy").save("/content/drive/MyDrive/Data/output/dfPersona")

#//Almacenamos los datos de CONTACTO
dfContacto.write.format("parquet").mode("overwrite").option("compression", "snappy").save("/content/drive/MyDrive/Data/output/dfContacto")

#//Almacenamos los datos de EMPRESA
dfEmpresa.write.format("parquet").mode("overwrite").option("compression", "snappy").save("/content/drive/MyDrive/Data/output/dfEmpresa")

#//Almacenamos los datos de TRANSACCION
dfTransaccion.write.format("parquet").mode("overwrite").option("compression", "snappy").save("/content/drive/MyDrive/Data/output/dfTransaccion")


## Ejemplo de procesamiento

In [25]:
#//Por ejemplo, alguien quisiera saber las edades y salarios de las personas que realizaron las transacciones

#//Leemos los datos de las transacciones
dfTransaccionLeida = spark.read.format("parquet").load("/content/drive/MyDrive/Data/output/dfTransaccion")

#//Mostramos los datos
dfTransaccionLeida.show()

#Leemos los datos de las personas
dfPersonaLeida = spark.read.format("parquet").load("/content/drive/MyDrive/Data/output/dfPersona")

#Mostramos los datos
dfPersonaLeida.show()

+----------+----------+------+----------+
|ID_PERSONA|ID_EMPRESA| MONTO|     FECHA|
+----------+----------+------+----------+
|        26|         5|2628.0|2021-01-23|
|        21|         9|4261.0|2021-01-23|
|        73|         7|1429.0|2021-01-23|
|        14|         5|3385.0|2021-01-23|
|        80|         4|3514.0|2021-01-23|
|        53|         9| 823.0|2021-01-23|
|        31|         2|3724.0|2021-01-23|
|        55|        10|3429.0|2021-01-23|
|        45|         4|4267.0|2021-01-23|
|        22|         9| 796.0|2021-01-23|
|        64|         4| 317.0|2021-01-23|
|        90|         9| 938.0|2021-01-23|
|        59|         8| 887.0|2021-01-23|
|        17|         5|2067.0|2021-01-23|
|        64|         7| 714.0|2021-01-23|
|        61|         5|2967.0|2021-01-23|
|        84|         1| 769.0|2021-01-23|
|        35|         3|3546.0|2021-01-23|
|        25|         2| 642.0|2021-01-23|
|        79|         3| 275.0|2021-01-23|
+----------+----------+------+----

In [38]:
#//Hacemos el cruce de datos
import pyspark.sql.functions as f

dfResultado = dfTransaccionLeida.alias("T").join(
  dfPersonaLeida.alias("P"),
  f.col("T.ID_PERSONA") == f.col("P.ID_PERSONA")
).select(
  "T.ID_PERSONA",
  "P.NOMBRE_PERSONA",
  "P.EDAD",
  "P.SALARIO",
  "T.ID_EMPRESA",
  "T.MONTO",
  "T.FECHA"
)

#//Mostramos los datos
dfResultado.show()


+----------+--------------+----+-------+----------+------+----------+
|ID_PERSONA|NOMBRE_PERSONA|EDAD|SALARIO|ID_EMPRESA| MONTO|     FECHA|
+----------+--------------+----+-------+----------+------+----------+
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|          null|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|       Brenden|  33|20549.0|         5|2628.0|2021-01-23|
|        26|        

### Tambien se podría convertir en un TempView del resultado para poder manipularlo con SQL
### Por lo general se le pone el mismo nombre

In [39]:

dfPersonaLeida.createOrReplaceTempView("dfPersona")

#Manipulemoslo desde SQL
dfResultadoPersona = spark.sql("SELECT * FROM dfPersona WHERE EDAD > 40")

#Vemos el contenido
dfResultadoPersona.show()

+----------+--------------+----+-------+
|ID_PERSONA|NOMBRE_PERSONA|EDAD|SALARIO|
+----------+--------------+----+-------+
|        73|         Fiona|  42| 9960.0|
|        14|         Allen|  59|16289.0|
|        80|         Ebony|  59| 3600.0|
|        31|         Rylee|  47|21591.0|
|        55|      Jennifer|  64|19013.0|
|        90|     Sigourney|  42| 9145.0|
|        17|          Omar|  60| 6851.0|
|        35|        Aurora|  54| 4588.0|
|        25|         Pearl|  52|14756.0|
|        79|        Philip|  51| 3919.0|
|        40|          Ross|  67|14285.0|
|        34|          Lila|  48|24305.0|
|        23|        Samson|  51| 8099.0|
|        95|         Jayme|  58|23975.0|
|        23|        Samson|  51| 8099.0|
|        70|          Suki|  43|12029.0|
|        27|          null|  55|13813.0|
|        79|        Philip|  51| 3919.0|
|         7|          Mark|  52| 8112.0|
|        34|          Lila|  48|24305.0|
+----------+--------------+----+-------+
only showing top